In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\shyam\\OneDrive\\Documents\\GitHub\\AI-Bot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shyam\\OneDrive\\Documents\\GitHub\\AI-Bot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [8]:
#extracted_data

In [9]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [34]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 37362


In [11]:
#text_chunks

In [12]:
#%pip uninstall sentence-transformers huggingface-hub -y
#%pip install sentence-transformers huggingface-hub --upgrade

In [13]:
#%pip install -U langchain-huggingface


In [14]:
from langchain_huggingface import HuggingFaceEmbeddings


In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
import os

def download_hugging_face_embeddings():
    local_model_path = "C:/Users/shyam/models/all-MiniLM-L6-V2"
    
    if not os.path.exists(local_model_path):
        raise FileNotFoundError(f"Model path does not exist: {local_model_path}")
    
    print(f"Loading model from: {local_model_path}")
    embeddings = HuggingFaceEmbeddings(model_name=local_model_path)
    print("Model loaded successfully.")
    
    return embeddings

embeddings = download_hugging_face_embeddings()


Loading model from: C:/Users/shyam/models/all-MiniLM-L6-V2


c:\Users\shyam\OneDrive\Documents\GitHub\AI-Bot\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model loaded successfully.


In [16]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
#query_result

In [35]:
from dotenv import load_dotenv
load_dotenv()

True

In [36]:
# Import the Pinecone library
from pinecone import Pinecone
import os

# Initialize a Pinecone client with your API key
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [28]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-hbpacab.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [29]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [38]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings, 
)

In [39]:
docsearch

In [40]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [41]:
retrieved_docs = retriever.invoke("What is Acne?")

In [42]:
retrieved_docs

[Document(id='c9854819-8252-4f54-b7dd-e1e59a9a0808', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='3162659e-9aec-481c-ae72-36fcc3dcee25', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nate